In [142]:
import numpy as np
import os
import pandas as pd
import csv
from matplotlib import pyplot as plt
import matplotlib.pyplot as pyplot
import PIL.Image as Image
from PIL import ImageOps
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.vq import vq, kmeans, whiten
import glob
import math
from sklearn.cluster import k_means

In [143]:
#[Larry's path] Brush_teeth_path = os.path.join(cur_dir,'data/Brush_teeth')
#[Titus's path] r'C:/Users/titus/Documents/CS498/hw4/HMP_Dataset/
cur_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
Brush_teeth_path = os.path.join(cur_dir,'data/Brush_teeth')
Climb_stairs_MODEL_path = os.path.join(cur_dir,'data/Climb_stairs') 
Comb_hair_path = os.path.join(cur_dir,'data/Comb_hair')
Descend_stairs_path = os.path.join(cur_dir,'data/Descend_stairs')
Drink_glass_MODEL_path = os.path.join(cur_dir,'data/Drink_glass')
Eat_meat_path = os.path.join(cur_dir,'data/Eat_meat') 
Eat_soup_path = os.path.join(cur_dir,'data/Eat_soup')
Getup_bed_MODEL_path = os.path.join(cur_dir,'data/Getup_bed')
liedown_bed_path = os.path.join(cur_dir,'data/liedown_bed')
Pour_water_MODEL_path = os.path.join(cur_dir,'data/Pour_water')
Sitdown_chair_MODEL_path = os.path.join(cur_dir,'data/Sitdown_chair')
Standup_chair_MODEL_path = os.path.join(cur_dir,'data/Standup_chair')
Use_telephone_path = os.path.join(cur_dir,'data/Use_telephone')
Walk_MODEL_path = os.path.join(cur_dir,'data/Walk')

In [144]:
path_list = [Brush_teeth_path, Climb_stairs_MODEL_path, Comb_hair_path, Descend_stairs_path, Drink_glass_MODEL_path, Eat_meat_path, Eat_soup_path, Getup_bed_MODEL_path, liedown_bed_path, Pour_water_MODEL_path, Sitdown_chair_MODEL_path, Standup_chair_MODEL_path, Use_telephone_path, Walk_MODEL_path]
df_list = []
testDF_list = []
for path in path_list:
    allFiles = glob.glob(path + "/*.txt")
    a = False
    for file_ in allFiles: #take every sample of one action in one single df
        if a == False:
            frame = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
            a = True
        else:
            df = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
            frame = pd.concat([frame,df])
    newdf = frame.iloc[0:32,:]
    newnp = newdf.as_matrix()
    vq_df = pd.DataFrame(np.array(newnp).resize((1,96)))
    
    #choseeing 20% to be the test data
    test_threshold = math.floor(len(frame.index)/32.0)*0.8 
    set_vqDF = False
    
    for i in range (1, math.floor(len(frame.index)/32.0)):
        newdf = frame.iloc[32*i:32*(i)+31, ]
        newnp = newdf.as_matrix()
        npreshape = pd.DataFrame(np.resize(newnp,(1,96)))
        if i < test_threshold:
            vq_df = pd.concat([vq_df,npreshape])
        else:
            if set_vqDF == False: 
                set_vqDF = True 
                test_df = pd.DataFrame(np.array(newnp).resize((1,96)))
            else: 
                test_df = pd.concat([test_df,npreshape])
    df_list.append(vq_df)
    testDF_list.append(test_df)

In [145]:
labels_list = ["Brush_teeth_path", "Climb_stairs_MODEL_path", "Comb_hair_path", "Descend_stairs_path", "Drink_glass_MODEL_path", "Eat_meat_path", "Eat_soup_path", "Getup_bed_MODEL_path", "liedown_bed_path", "Pour_water_MODEL_path", "Sitdown_chair_MODEL_path", "Standup_chair_MODEL_path", "Use_telephone_path", "Walk_MODEL_path"]
classifier_y = []
classifier_x = []
init = True 
init_first_layer = [True]*40
first_layer = [pd.DataFrame(np.array(newnp).resize((1,96)))]*40
for act in range(len(labels_list)): 
    whitene = whiten(df_list[act])
    centroid, label, inertia = k_means(whitene.astype(float), 40)
    for i in range(label.shape[0]): 
        #row1 = (whitene.iloc[i]).as_matrix()
        row2 = pd.DataFrame(whitene[i].reshape((1,96)))
        c = label[i]
        if init_first_layer[c] == True:
            init_first_layer[c] = False
            first_layer[c] =row2
        else:
            first_layer[c] = pd.concat([first_layer[c],row2])
    histogram_vector = []
    second_layer = [0]*480
    for i in range(40):
        cur_cluster = first_layer[i] 
        whitene = whiten(cur_cluster)
        if len(whitene) < 12: 
            label = range(len(whitene))
            num_element = len(whitene)
        else:
            centroid, label, inertia = k_means(whitene.astype(float), 12)
            num_element = label.shape[0]
        for j in range(num_element): 
            c = i*12 + label[j]
            second_layer[c] = second_layer[c] + 1
            histogram_vector.append(c)
            
    if init == True: 
        classifier_x = pd.DataFrame(np.array(second_layer).reshape((1,480)))
        init = False
    else: 
        classifier_x = pd.concat([classifier_x, pd.DataFrame(np.array(second_layer).reshape(1,480))])

    classifier_y.append(act)
    
#     n, bins, patches = plt.hist(histogram_vector, bins=480,facecolor='green')
#     plt.xlabel('Centroids')
#     plt.ylabel('Frequency')
#     plt.title(labels_list[act])
#     plt.show()



/Users/poonl/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/cluster/vq.py:141: RuntimeWarning: Some columns have standard deviation zero. The values of these columns will not change.
  RuntimeWarning)


In [146]:
classifier_x

,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,479
0,1,1,1,1,1,1,1,1,1,1,...,4,1,3,7,10,10,1,3,1,1
0,10,2,22,1,1,1,2,12,1,1,...,18,3,18,1,3,8,6,6,1,1
0,17,8,1,3,22,2,2,3,14,2,...,29,9,22,1,14,3,4,1,1,1
0,3,22,2,11,2,2,4,13,22,3,...,2,5,27,1,4,32,1,1,13,1
0,32,23,17,2,11,2,21,2,3,2,...,18,4,1,32,7,1,1,17,17,8
0,2,32,21,18,10,2,11,23,2,4,...,19,22,7,1,10,21,1,8,9,4
0,3,28,32,20,3,5,2,11,16,7,...,13,25,2,5,28,1,1,6,5,13
0,20,32,28,10,15,3,3,11,3,7,...,1,19,2,11,10,34,3,1,4,14
0,10,16,32,39,10,3,2,6,3,11,...,19,5,2,45,7,20,1,30,4,3
0,5,32,16,39,10,5,18,3,28,17,...,21,13,28,2,11,4,1,32,3,32


In [147]:
test_y = []
init = True
for act in range(len(labels_list)): 
    whitene = whiten(testDF_list[act])
    centroid, label, inertia = k_means(whitene.astype(float), 40)
    first_layer = [0]*40
    init_test_clusters = [True]*40
    for i in range(label.shape[0]): 
        row2 = pd.DataFrame(whitene[i].reshape((1,96)))
        c = label[i]
        if init_test_clusters[c] == True: 
            init_test_clusters[c] = False
            first_layer[c] = row2
        else: 
            first_layer[c] = pd.concat([first_layer[c],row2])
        #set up for second layer
        histogram_vector = []
        second_layer = [0]*480
    for i in range(40):
        cur_cluster = first_layer[i] 
        whitene = whiten(cur_cluster)
        if len(whitene) < 12: 
            label = range(len(whitene))
            num_element = len(whitene)
        else:
            centroid, label, inertia = k_means(whitene.astype(float), 12)
            num_element = label.shape[0]
        for j in range(num_element): 
            c = i*12 + label[j]
            second_layer[c] = second_layer[c] + 1
            histogram_vector.append(c)

    if init == True: 
        test_x = pd.DataFrame(np.array(second_layer).reshape((1,480)))
        init = False
    else: 
        test_x = pd.concat([test_x, pd.DataFrame(np.array(second_layer).reshape(1,480))])
    test_y.append(act)

/Users/poonl/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/cluster/vq.py:141: RuntimeWarning: Some columns have standard deviation zero. The values of these columns will not change.
  RuntimeWarning)


In [148]:
test_x

,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,479
0,1,1,3,1,1,2,1,1,1,1,...,1,1,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,5,1,2,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [149]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

rf = RandomForestClassifier(n_estimators=10,max_depth=16 )
y_pred = rf.fit(classifier_x, classifier_y).predict(test_x)
result = sum(np.in1d(test_y,y_pred))/len(y_pred)

In [152]:
result

0.07142857142857142